In [19]:
import numpy as np 
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import os
import random
#Importing data(iris)
from sklearn.datasets import load_iris

In [20]:
# encoding_words = 'X5'
# pattern = '1'
# number = '1'
# salt_words = ['LICLO4','LICL','NO']
# fiber_words = ['5-4','8-1','6']
# path_folder = f'C:\\Users\\laboratorio20\\codigos_python\\fiber_experiments\\reservoir_iris\\reservoir_pattern_{pattern}\\reservoir_{number}\\post_fiber_growth_electrode_#{fiber_words[2]}'
# os.chdir(path=path_folder)


In [21]:
# #encoding part data treatment
# encoding_words = ['encoding=X5','encoding=No', 'encoding=MinMaxScaler']
# pattern = '3'
# number = '3'
# salt_words = ['LICLO4','LICL','NO']
# fiber_words = '5-4_8-1_6'


# path_folder = f'C:\\Users\\laboratorio20\\codigos_python\\fiber_experiments\\reservoir_iris\\reservoir_pattern_{pattern}\\reservoir_{number}\\encoding_test'
# os.chdir(path=path_folder)


In [39]:
#encoding part data treatment
encoding_words = ['encoding=X5','encoding=No', 'encoding=MinMaxScaler']
pattern = '10'
salt_words = ['LICLO4','LICL','NO']
fiber_words = '7-2_6-3_8-1_5-4'


path_folder = f'C:\\Users\\laboratorio20\\codigos_python\\fiber_experiments\\reservoir_iris\\reservoir_pattern_{pattern}'
os.chdir(path=path_folder)


In [23]:
# #encoding part data treatment
# encoding_words = ['encoding=X5','encoding=No', 'encoding=MinMaxScaler']
# pattern = "10"

# salt_words = 'NO'
# fiber_words = 'Resistor Mesh 2'


# path_folder = f'C:\\Users\\laboratorio20\\codigos_python\\fiber_experiments\\reservoir_iris\\reservoir_pattern_{pattern}'
# os.chdir(path=path_folder)


In [24]:
random.seed(7)
np.random.seed(7)

In [25]:
def accuracy_elementwise(ypred, ytrue):
    # compute accuracy at each time step
    yp = ypred.reshape(-1, ytrue.shape[2]).argmax(axis=1)         #argmax(axis =1) is the "winner takes all" approach from the paper "hands on Reservoir computing"
    yt = ytrue.reshape(-1, ytrue.shape[2]).argmax(axis=1)
    return np.sum(yp == yt) / yp.shape[0]




def accuracy(ypred, ytrue):
    # compute accuracy by averaging the predicted class over the whole time series
    yp = ypred.reshape(ytrue.shape).mean(axis=1).argmax(axis=1)
    yt = ytrue[:,0,:].argmax(axis=1)
    return np.sum(yp == yt) / yp.shape[0]

In [26]:
iris = load_iris()
iris_df = pd.DataFrame(iris.data)

iris_df['class']=iris.target

iris_df.columns=['sepal_len', 'sepal_wid', 'petal_len', 'petal_wid', 'class']
iris_df.dropna(how="all", inplace=True) # remove any empty lines

#selecting only first 4 columns as they are the independent(X) variable
# any kind of feature selection or correlation analysis should be first done on these

print(iris_df)

     sepal_len  sepal_wid  petal_len  petal_wid  class
0          5.1        3.5        1.4        0.2      0
1          4.9        3.0        1.4        0.2      0
2          4.7        3.2        1.3        0.2      0
3          4.6        3.1        1.5        0.2      0
4          5.0        3.6        1.4        0.2      0
..         ...        ...        ...        ...    ...
145        6.7        3.0        5.2        2.3      2
146        6.3        2.5        5.0        1.9      2
147        6.5        3.0        5.2        2.0      2
148        6.2        3.4        5.4        2.3      2
149        5.9        3.0        5.1        1.8      2

[150 rows x 5 columns]


In [27]:
# normalizing DataFrame columns between +1 and -1. Each variable column is normalized between 0 and 1.


# # Create the scaler
# scaler = MinMaxScaler()

# # Fit and transform the data

# iris_df_normalized = pd.DataFrame(scaler.fit_transform(iris_df), columns=iris_df.columns)


#one hot encoding target variables
iris_df_one_hot_encoded = pd.get_dummies(iris_df,columns=['class'],drop_first=False,dtype='int')



iris_df_one_hot_encoded['petal_wid'] = iris_df_one_hot_encoded['petal_wid'] * 5    #multiplying petal wid column by 5 to increase its frequency

# arr = iris_df.iloc[0,[0,1,2,3]]

print(iris_df_one_hot_encoded)

     sepal_len  sepal_wid  petal_len  petal_wid  class_0  class_1  class_2
0          5.1        3.5        1.4        1.0        1        0        0
1          4.9        3.0        1.4        1.0        1        0        0
2          4.7        3.2        1.3        1.0        1        0        0
3          4.6        3.1        1.5        1.0        1        0        0
4          5.0        3.6        1.4        1.0        1        0        0
..         ...        ...        ...        ...      ...      ...      ...
145        6.7        3.0        5.2       11.5        0        0        1
146        6.3        2.5        5.0        9.5        0        0        1
147        6.5        3.0        5.2       10.0        0        0        1
148        6.2        3.4        5.4       11.5        0        0        1
149        5.9        3.0        5.1        9.0        0        0        1

[150 rows x 7 columns]


In [28]:
iris_df_one_hot_encoded = iris_df_one_hot_encoded.to_numpy()

In [29]:
# iris_df_normalized_one_hot_encoded = iris_df_normalized_one_hot_encoded.to_numpy()

timesteps = np.arange(0,1,0.01)

u = np.array([ np.vstack([(np.sin(timesteps*(np.pi)*pt[i]))/3 for i in range(4)]).T
    for pt in iris_df_one_hot_encoded[:,:4]
])
y = np.array([iris_df_one_hot_encoded[:,4:]] * len(timesteps)).swapaxes(0, 1)



In [30]:
#creating sampling indexes
sample = lambda a, b: random.sample(range(a, b), 40)
train_idxs = sample(0, 50) + sample(50, 100) + sample(100, 150)           #120 train samples
test_idxs = [idx for idx in range(150) if idx not in train_idxs]          #30 test samples


In [31]:
#splitting data in train and test sets
u_train = u[train_idxs]
y_train = y[train_idxs]
u_test = u[test_idxs]
y_test = y[test_idxs]
print(train_idxs)

[20, 9, 25, 41, 3, 4, 34, 6, 23, 37, 45, 32, 13, 2, 5, 27, 26, 44, 15, 36, 17, 40, 1, 33, 18, 39, 7, 49, 22, 47, 43, 21, 12, 19, 46, 0, 8, 30, 38, 42, 59, 84, 57, 86, 69, 85, 93, 61, 56, 87, 96, 62, 73, 91, 94, 54, 53, 63, 81, 71, 67, 82, 74, 60, 64, 68, 75, 92, 99, 97, 55, 70, 52, 80, 65, 90, 95, 72, 77, 98, 138, 104, 107, 132, 126, 110, 121, 109, 131, 145, 102, 148, 135, 136, 120, 143, 122, 141, 129, 139, 140, 130, 108, 115, 133, 134, 128, 101, 142, 137, 118, 114, 124, 112, 111, 100, 147, 105, 123, 117]


In [32]:

files = os.listdir(path=path_folder) 
train_states_list = list()
test_states_list = list()
for i in files:
    if ('train_states' in i) and ('.pdf') not in i:
        train_states_list.append(i)
    if ('test_states' in i) and ('.pdf') not in i:
        test_states_list.append(i)


train_test_states_list = list(zip(train_states_list,test_states_list))


for i in train_test_states_list:
    print(i)

('iris_dataset_train_states_7-2_6-3_8-1_5-4_FIBER_LICLO4_ELECTROLYTE_RES_CHIP_0010_CE1_#1_encoding=MinMaxScaler_2025_02_21.txt', 'iris_dataset_test_states_7-2_6-3_8-1_5-4_FIBER_LICLO4_ELECTROLYTE_RES_CHIP_0010_CE1_#1_encoding=MinMaxScaler_2025_02_21.txt')
('iris_dataset_train_states_7-2_6-3_8-1_5-4_FIBER_LICLO4_ELECTROLYTE_RES_CHIP_0010_CE1_#1_encoding=No_Encoding_2025_02_21.txt', 'iris_dataset_test_states_7-2_6-3_8-1_5-4_FIBER_LICLO4_ELECTROLYTE_RES_CHIP_0010_CE1_#1_encoding=No_Encoding_2025_02_21.txt')
('iris_dataset_train_states_7-2_6-3_8-1_5-4_FIBER_LICLO4_ELECTROLYTE_RES_CHIP_0010_CE1_#1_encoding=X5_2025_02_21.txt', 'iris_dataset_test_states_7-2_6-3_8-1_5-4_FIBER_LICLO4_ELECTROLYTE_RES_CHIP_0010_CE1_#1_encoding=X5_2025_02_21.txt')
('iris_dataset_train_states_7-2_6-3_8-1_5-4_FIBER_LICLO4_ELECTROLYTE_RES_CHIP_0010_CE3_#2_encoding=MinMaxScaler_2025_02_21.txt', 'iris_dataset_test_states_7-2_6-3_8-1_5-4_FIBER_LICLO4_ELECTROLYTE_RES_CHIP_0010_CE3_#2_encoding=MinMaxScaler_2025_02_21.txt'

In [33]:
def compute_wout(X, Y, T_washout=0, alpha=1e-3):
    n_samples, n_timesteps, _ = Y.shape
    
    X = X.reshape((n_samples, n_timesteps, -1))[:, T_washout:, :]
    X = np.concatenate(X)
    Y = np.concatenate(Y[:, T_washout:, :])
    R = X.T @ X
    P = X.T @ Y
    return np.linalg.inv(R + alpha * np.eye(X.shape[1])) @ P

In [34]:
washout_list = [i for i in range(0,31,3)]
alfa_list = [0,1e-5,1e-4,1e-3,1e-2,1e-1,1,10]



In [35]:
# #Growth and post growth data treatment
# for k in washout_list:
#     for j in alfa_list:

#         for i in train_test_states_list:
#             X_train = np.loadtxt(i[0],delimiter='\t',skiprows=1,usecols=[1,2,3,4])
#             X_test = np.loadtxt(i[1],delimiter='\t',skiprows=1,usecols=[1,2,3,4])
#             wout_matrix = compute_wout(X=X_train,Y=y_train,T_washout=k,alpha=j)
#             y_pred_test = X_test @ wout_matrix
#             file_words =  i[0].split('_')
#             y_pred_test_series = pd.Series(y_pred_test.reshape(y_test.shape).mean(axis=1).argmax(axis=1), name='Predicted')
#             y_test_series = pd.Series(y_test[:,0,:].argmax(axis=1), name='Actual')
            
#             df_confusion = pd.crosstab(y_test_series, y_pred_test_series)

            
#             for salt in salt_words:
#                 if salt in file_words:

#                     if fiber_words[1] in file_words and fiber_words[2] in file_words:
#                         print(f'{fiber_words[0]}_{fiber_words[1]}_{fiber_words[2]}\t{salt}\tx5\t{pattern}\t{number}\t{file_words[-3]}-{file_words[-2]}-{file_words[-1].replace('.txt','')}\t{j}\t{k}\t{df_confusion.columns.values}\t{len(df_confusion.columns.values)}\t{accuracy(y_pred_test, y_test):.2%}')

#                     else:            
#                         print(f'{fiber_words[0]}_{fiber_words[1]}\t{salt}\tx5\t{pattern}\t{number}\t{file_words[-3]}-{file_words[-2]}-{file_words[-1].replace('.txt','')}\t{j}\t{k}\t{df_confusion.columns.values}\t{len(df_confusion.columns.values)}\t{accuracy(y_pred_test, y_test):.2%}')



In [44]:
#encoding data treatment

for k in washout_list:
    for j in alfa_list:

        for i in train_test_states_list:
            X_train = np.loadtxt(i[0],delimiter='\t',skiprows=1,usecols=[1,2,3,4])
            X_test = np.loadtxt(i[1],delimiter='\t',skiprows=1,usecols=[1,2,3,4])
            wout_matrix = compute_wout(X_train,y_train,k,j)
            y_pred_test = X_test @ wout_matrix
            file_words =  i[0].split('_')
            y_pred_test_series = pd.Series(y_pred_test.reshape(y_test.shape).mean(axis=1).argmax(axis=1), name='Predicted')
            y_test_series = pd.Series(y_test[:,0,:].argmax(axis=1), name='Actual')
            # print(file_words)
            df_confusion = pd.crosstab(y_test_series, y_pred_test_series)

            for salt in salt_words:
                
                if salt in file_words:

                    if encoding_words[0] in file_words:
                        print(f'{fiber_words}\t{salt}\tx5\t{pattern}\t{file_words[15].replace('#','')}\t{file_words[-3]}-{file_words[-2]}-{file_words[-1].replace('.txt','')}\t{j}\t{k}\t{df_confusion.columns.values}\t{len(df_confusion.columns.values)}\t{accuracy(y_pred_test, y_test):.2%}')

                    elif encoding_words[1] in file_words:
                        print(f'{fiber_words}\t{salt}\tNO\t{pattern}\t{file_words[15].replace('#','')}\t{file_words[-3]}-{file_words[-2]}-{file_words[-1].replace('.txt','')}\t{j}\t{k}\t{df_confusion.columns.values}\t{len(df_confusion.columns.values)}\t{accuracy(y_pred_test, y_test):.2%}')
                    else:            
                        print(f'{fiber_words}\t{salt}\tMinMaxScaler\t{pattern}\t{file_words[15].replace('#','')}\t{file_words[-3]}-{file_words[-2]}-{file_words[-1].replace('.txt','')}\t{j}\t{k}\t{df_confusion.columns.values}\t{len(df_confusion.columns.values)}\t{accuracy(y_pred_test, y_test):.2%}')



7-2_6-3_8-1_5-4	LICLO4	MinMaxScaler	10	1	2025-02-21	0	0	[0 1 2]	3	60.00%
7-2_6-3_8-1_5-4	LICLO4	NO	10	1	2025-02-21	0	0	[0 2]	2	30.00%
7-2_6-3_8-1_5-4	LICLO4	x5	10	1	2025-02-21	0	0	[0]	1	33.33%
7-2_6-3_8-1_5-4	LICLO4	MinMaxScaler	10	2	2025-02-21	0	0	[0 1 2]	3	56.67%
7-2_6-3_8-1_5-4	LICLO4	NO	10	2	2025-02-21	0	0	[1]	1	33.33%
7-2_6-3_8-1_5-4	LICLO4	x5	10	2	2025-02-21	0	0	[0]	1	33.33%
7-2_6-3_8-1_5-4	LICLO4	MinMaxScaler	10	3	2025-02-21	0	0	[0 1 2]	3	63.33%
7-2_6-3_8-1_5-4	LICLO4	NO	10	3	2025-02-21	0	0	[0 1 2]	3	40.00%
7-2_6-3_8-1_5-4	LICLO4	x5	10	3	2025-02-21	0	0	[0 2]	2	43.33%
7-2_6-3_8-1_5-4	LICL	MinMaxScaler	10	1	2025-02-21	0	0	[0 1 2]	3	63.33%
7-2_6-3_8-1_5-4	LICL	NO	10	1	2025-02-21	0	0	[0 1]	2	53.33%
7-2_6-3_8-1_5-4	LICL	x5	10	1	2025-02-21	0	0	[0]	1	33.33%
7-2_6-3_8-1_5-4	LICL	MinMaxScaler	10	2	2025-02-21	0	0	[0 1 2]	3	63.33%
7-2_6-3_8-1_5-4	LICL	NO	10	2	2025-02-21	0	0	[0 1]	2	36.67%
7-2_6-3_8-1_5-4	LICL	x5	10	2	2025-02-21	0	0	[0 2]	2	33.33%
7-2_6-3_8-1_5-4	LICL	MinMaxScaler	10	3	2025

In [37]:
# #encoding data treatment

# for k in washout_list:
#     for j in alfa_list:

#         for i in train_test_states_list:
#             X_train = np.loadtxt(i[0],delimiter='\t',skiprows=1,usecols=[1,2,3,4])
#             X_test = np.loadtxt(i[1],delimiter='\t',skiprows=1,usecols=[1,2,3,4])
#             wout_matrix = compute_wout(X_train,y_train,k,j)
#             y_pred_test = X_test @ wout_matrix
#             file_words =  i[0].split('_')
#             # print(file_words)
#             y_pred_test_series = pd.Series(y_pred_test.reshape(y_test.shape).mean(axis=1).argmax(axis=1), name='Predicted')
#             y_test_series = pd.Series(y_test[:,0,:].argmax(axis=1), name='Actual')
            
#             df_confusion = pd.crosstab(y_test_series, y_pred_test_series)
                

#             if encoding_words[0] in file_words:
#                 print(f'{fiber_words}\t{salt_words}\tx5\t4\t{file_words[6].replace('#','')}\t{file_words[-3]}-{file_words[-2]}-{file_words[-1].replace('.txt','')}\t{j}\t{k}\t{df_confusion.columns.values}\t{len(df_confusion.columns.values)}\t{accuracy(y_pred_test, y_test):.2%}')

#             elif encoding_words[1] in file_words:
#                 print(f'{fiber_words}\t{salt_words}\tNO\t4\t{file_words[6].replace('#','')}\t{file_words[-3]}-{file_words[-2]}-{file_words[-1].replace('.txt','')}\t{j}\t{k}\t{df_confusion.columns.values}\t{len(df_confusion.columns.values)}\t{accuracy(y_pred_test, y_test):.2%}')
#             else:            
#                 print(f'{fiber_words}\t{salt_words}\tMinMaxScaler\t4\t{file_words[6].replace('#','')}\t{file_words[-3]}-{file_words[-2]}-{file_words[-1].replace('.txt','')}\t{j}\t{k}\t{df_confusion.columns.values}\t{len(df_confusion.columns.values)}\t{accuracy(y_pred_test, y_test):.2%}')



In [38]:
# #encoding data treatment

# for k in washout_list:
#     for j in alfa_list:

#         for i in train_test_states_list:
#             X_train = np.loadtxt(i[0],delimiter='\t',skiprows=1,usecols=[1,2,3,4])
#             X_test = np.loadtxt(i[1],delimiter='\t',skiprows=1,usecols=[1,2,3,4])
#             wout_matrix = compute_wout(X_train,y_train,k,j)
#             y_pred_test = X_test @ wout_matrix
#             file_words =  i[0].split('_')
#             # print(file_words)
#             y_pred_test_series = pd.Series(y_pred_test.reshape(y_test.shape).mean(axis=1).argmax(axis=1), name='Predicted')
#             y_test_series = pd.Series(y_test[:,0,:].argmax(axis=1), name='Actual')
            
#             df_confusion = pd.crosstab(y_test_series, y_pred_test_series)
                

#             if encoding_words[0] in file_words:
#                 print(f'{fiber_words}\t{salt_words}\tx5\t4\t{file_words[6].replace('#','')}\t{file_words[-3]}-{file_words[-2]}-{file_words[-1].replace('.txt','')}\t{j}\t{k}\t{df_confusion.columns.values}\t{len(df_confusion.columns.values)}\t{accuracy(y_pred_test, y_test):.2%}')

#             elif encoding_words[1] in file_words:
#                 print(f'{fiber_words}\t{salt_words}\tNO\t4\t{file_words[6].replace('#','')}\t{file_words[-3]}-{file_words[-2]}-{file_words[-1].replace('.txt','')}\t{j}\t{k}\t{df_confusion.columns.values}\t{len(df_confusion.columns.values)}\t{accuracy(y_pred_test, y_test):.2%}')
#             else:            
#                 print(f'{fiber_words}\t{salt_words}\tMinMaxScaler\t4\t{file_words[6].replace('#','')}\t{file_words[-3]}-{file_words[-2]}-{file_words[-1].replace('.txt','')}\t{j}\t{k}\t{df_confusion.columns.values}\t{len(df_confusion.columns.values)}\t{accuracy(y_pred_test, y_test):.2%}')
